In [2]:
import re

In [8]:
text = "རྗེ་བཙུན་བླ་མ་དམིགས་པ་མེད་པའི་ཐུགས་རྗེ་ཆེན་པོ་དང་ལྡན་པ་རྣམས་ལ་ཕྱག་འཚལ་ལོ། །རིག་འཛིན"
lexicon = {'རྗེ་བཙུན' : '', 'བླ་མ' : '', 'དམིགས་པ' : '', 'མེད་པ' : '', 'ཐུགས་རྗེ' : '', 'ཆེན་པོ' : '', 'དང' : '', 'ལྡན་པ' : '', 'རྣམས' : '', '། ' : '', '། །' : '', '།' : '', 'ལ' : '', 'ཕྱག' : '', 'འཚལ' : ''}
merged_part = {'ར''ས''འི''འམ''འང'}

l1 = []
l2 = []

def isWord(syls):
    maybe = '་'.join(syls)
    final = False
    if maybe in lexicon:
        final = True
    elif re.sub(r'(ར|ས|འི|འམ|འང)$', '', maybe) in lexicon:
        final = True
    return final

def process(list1, list2, num):
    word = '་'.join(l1[:num])
    if word not in lexicon:
        maybe = re.split(r'(ར|ས|འི|འམ|འང)$', word)[:-1]
        list2.append(maybe[0])
        list2.append(maybe[1]+'་')
        del list1[:num]
    else:
        list2.append(word+"་")
        del list1[:num]

text1 = re.sub(r"([།|༎|༏|༐|༑|༔|\s]+)", "་\g<1>་", text)
print(text1)
l1 = re.split(r"་+", text1)
        
while len(l1) > 0:
    if   isWord(l1[:4]): process(l1, l2, 4)
    elif isWord(l1[:3]): process(l1, l2, 3)
    elif isWord(l1[:2]): process(l1, l2, 2)
    elif isWord(l1[:1]): process(l1, l2, 1)
    else:
        l2.append('་'.join(l1[:1])+"་*")
        del l1[:1]

yo = " ".join(l2)
print(yo)

རྗེ་བཙུན་བླ་མ་དམིགས་པ་མེད་པའི་ཐུགས་རྗེ་ཆེན་པོ་དང་ལྡན་པ་རྣམས་ལ་ཕྱག་འཚལ་ལོ་། །་རིག་འཛིན
རྗེ་བཙུན་ བླ་མ་ དམིགས་པ་ མེད་པ འི་ ཐུགས་རྗེ་ ཆེན་པོ་ དང་ ལྡན་པ་ རྣམས་ ལ་ ཕྱག་ འཚལ་ ལོ་* ། །་ རིག་* འཛིན་*


In [19]:

    print(num)

4
3
2
1
